### Setup & Functions

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import cv2
import matplotlib.pyplot as plt
import random
from keras.backend import clear_session
from keras.backend import get_session
import time
from datetime import timedelta 

import math
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
os.chdir("..\\asset") # change working directory

In [4]:
emotionList = ('Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt')
TRAINANNOTATIONPATH = "train_set\\annotations\\"
TRAINIMAGEPATH = "train_set\\images\\"
TESTANNOTATIONPATH = "val_set\\annotations\\"
TESTIMAGEPATH = "val_set\\images\\"

def LoadAllImageNames(filePath, limit = 0, catLimit = []):
  """Load the images names and label in tuple format (label, image name)

  Args:
      filePath (str): directory of image folder
      limit (int, optional): max number of image to load. Defaults to 0.
      catLimit (list<int>, optional): array of image count limit for each class. Defaults to []*8.

  Returns:
      list<str>: shuffled list of image names
  """
  limitCounter = [0,0,0,0,0,0,0,0]
  dataSet = []
  loadCounter = 0
  for file in os.scandir(filePath):
    if (limit > 0 and loadCounter > limit):
      break
  
    name = file.name[:-4] # file name w/o file extension
    data = np.load("{}{}_exp.npy".format(TRAINANNOTATIONPATH if filePath == TRAINIMAGEPATH else TESTANNOTATIONPATH, name)) # 
    label = int(data.item(0))

    if len(catLimit) > 0 and limitCounter[label] >= catLimit[label]:
        continue
    limitCounter[label] += 1
    loadCounter += 1

    dataSet.append((label, file.name))
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
  
  print("Total images loaded: ", loadCounter)
  print("Images Loaded: ", limitCounter)
  random.shuffle(dataSet)
  return dataSet

# Load the pixels of a picture to numpy.ndarray format. false for test set, true for training set
# Return image in RGB format
def LoadImage(imagePath, imageName, normalize = True):
  """Load image using numpy

  Args:
      imagePath (str): image path 
      imageName (str): image name
      normalize (bool, optional): To normalize image or not. Defaults to True.

  Returns:
      numpy array: x,y,3 array
  """
  # print("{}{}{}".format(os.getcwd(), "\\"+imagePath, imageName))
  image_array = cv2.imread("{}{}{}".format(os.getcwd(), "\\"+imagePath, imageName))
  image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
  if normalize:
    image_array = image_array/255
    
  return image_array

# Extract the daata from 0 to amount from list and return it
def CropData(list, amount):
  if (len(list) < amount):
    amount = len(list)
  croppedList = list[:amount]
  del list[:amount]
  return croppedList

def LoadImages(imagePath, list):
  label = []
  data = []
  count = 0
  for entries in list:
    # try:
      image = LoadImage(imagePath, entries[1])
      data.append(image) 
      label.append(entries[0]) 
    # except: 
      # print("Failed to load training image: ", entries[1])
  npLabel = np.array(label) 
  npData = np.array(data) 
  return npLabel, npData 

def InitializeModel():
  pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  # Adding 3 more layers to output side
  final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(64)(final_output)
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  new_model = keras.Model(inputs = base_input, outputs = final_output)
  # new_model.summary()
  return new_model

def ConvertToGray(image):
  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  return image

def ScaleImage(image, width):
  ratio = image.shape[1]/width
  image = cv2.resize(image, (width, int(image.shape[0]/ratio)))
  return image

def DetectFace(image):
  face_roi = np.ndarray(1)
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = faceCascade.detectMultiScale(grayImage, 1.3, 5)
  for x,y,w,h in faces:
    roi_gray = grayImage[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if (len(facess) == 0):
      print("Face not detected")
    else:
      for (ex,ey,ew,eh) in facess:
        face_roi = roi_color[ey:ey+eh, ex:ex+ew]
  return face_roi

def ConvertToInput(image):
  input = ScaleImage(image, 224)
  input = np.expand_dims(input, axis = 0) ## to add fourth dimension to fit model input
  input = input/255
  return input

def GetResult(model, input):
  Predictions = model.predict(input)
  print(Predictions)
  result = np.argmax(Predictions)
  return emotionList[result]

def printDataSetLabels(dataSet):
    counterList = list(range(8))
    for label, name in dataSet:
        counterList[label] += 1
    print(counterList)
    
# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    # try:
    #     del classifier # this is from global space - change this as you need
    # except:
    #     pass

    # use the same config as you used to create the session
    tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9, visible_device_list="0")
    
def TestModel(model, testSetData):
  # test model
  lostSum = 0
  accuracySum = 0
  count = 0
  while(len(testSetData) != 0):
    # training data
    try:
      croppedList = CropData(testSetData, 100)
      label, data = LoadImages(TESTIMAGEPATH, croppedList)
      result = model.evaluate(data, label, batch_size = 1)
      lostSum += result[0]
      accuracySum += result[1]
    except:
      print("Failed to test data")

    count += 1
    reset_keras()
  
  return lostSum/count, accuracySum/count

class TimeHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.times = []

  def on_epoch_begin(self, batch, logs={}):
    self.epoch_time_start = time.time()

  def on_epoch_end(self, batch, logs={}):
    self.times.append(time.time() - self.epoch_time_start)

  def AverageTime(self):
    sum = 0
    for time in self.times:
      sum += time
    return sum/len(self.times)



### Data Processing

In [5]:
mainTrainSet = LoadAllImageNames(TRAINIMAGEPATH, catLimit=[4000]*8)
mainTestSet = LoadAllImageNames(TESTIMAGEPATH)

Files loaded:10000
Files loaded:20000
Files loaded:30000
Total images loaded:  31553
Images Loaded:  [4000, 4000, 4000, 4000, 4000, 3803, 4000, 3750]
Total images loaded:  3999
Images Loaded:  [500, 500, 500, 500, 500, 500, 500, 499]


In [4]:
# reset learning rate callback to counter ReduceLROnPlateau
class ResetLR(tf.keras.callbacks.Callback):
    def __init__(self, learningRate):
        self.default_lr = learningRate
    def on_train_begin(self, logs={}):
        previous_lr = self.model.optimizer.lr.read_value()
        if previous_lr != self.default_lr:
            print("Resetting learning rate from {} to {}".format(previous_lr, self.default_lr))
            self.model.optimizer.lr.assign(self.default_lr)

### Model prepping
Change the pretrained model and layers here

In [6]:
def DoubleLayerTrainAndTest(modelName, epoch, learningRate, batchSize, firstLayerSize, secondLayerSize):
  """Train and test a model. This function's input should be customized to accept different parameters, function body should be customized to these parameters to train models of variety of paramters.

  Args:
      modelName (str): name of model
  """
  pretrained_model = tf.keras.applications.MobileNetV3Small(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  # Freeze layer
  pretrained_model.trainable = False
  for layer in pretrained_model.layers:
    assert layer.trainable == False
    layer.trainable = False

  #-------------------------------------------------
  # Customizing layers
  #-------------------------------------------------
  # Adding 3 more layers to output side
  final_output = layers.Dense(firstLayerSize)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(secondLayerSize)(final_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  model = keras.Model(inputs = base_input, outputs = final_output)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = keras.optimizers.Adam(lr = learningRate), metrics=["accuracy"])

  # ------------------ Callbacks ------------------
  # earlyStop = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights = True)
  # earlyStopAcc = EarlyStopping(monitor='val_accuracy', patience = 1, restore_best_weights = True)
  # reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 1, min_lr = 0.0002)
  # reset_lr = ResetLR(learningRate)
    
  # ------------------ Training Model ------------------
  time_callback = TimeHistory()
  dataSet = mainTrainSet.copy()

  # Initializing model fit params
  # batchSize = 4
  imgPerIter = batchSize*32

  count = 0
  iter = 0
  while(len(dataSet) != 0):
    iter += 1
    croppedList = []
    tensorboard = TensorBoard(log_dir = "logs/{}_it{}".format(modelName, iter))
    try:
      print("loading image") # Crop and load images in
      croppedList = CropData(dataSet, imgPerIter)
      label, data = LoadImages(TRAINIMAGEPATH, croppedList)
      # Training model
      model.fit(data, label, epochs = epoch, validation_split = 0.2, batch_size = batchSize, callbacks = [time_callback, tensorboard])
      print(time_callback.AverageTime())
    except RuntimeError as e:
      print(e)
    
    # Saving weights
    model.save_weights('{}.h5'.format(modelName))
    
    # Print summary of current iteration
    count = count + len(croppedList)
    timeLeft = len(dataSet)/imgPerIter * time_callback.AverageTime()
    print("trained image count: ", count)
    print("Images Left: ", len(dataSet))
    print("Estimated completion time: ", str(timedelta(seconds=timeLeft)))
    
    reset_keras()
  print("---------------- {} Training Completed ------------------".format(modelName))

  # ---------------- Testing Model -----------------
  # initialise image names and label

  # test model
  loss, accuracy = TestModel(model, mainTestSet.copy())
  
  # Saving test informations
  modelSummary = '''
  model name: {}
  average lost: {}
  average accuracy: {}
            '''.format(modelName, loss, accuracy)
      
  print("==========FINISH TESTING===========\n{}".format(modelSummary))
  
  file = open("training_summaries.txt", "a")
  file.write(modelSummary + "\n\n")

In [ ]:
batchSizes = [48, 64, 96, 128]
firstLayerSizes = [384, 516, 772]
secondLayerSizes = [32, 48, 64, 96, 128]
epoch = 13
learningRate = 0.001

for batchSize in batchSizes:
    for firstLayerSize in firstLayerSizes:
        for secondLayerSize in secondLayerSizes:
            if firstLayerSize <= secondLayerSize:
                continue
            modelName = "freezeCNN_{}epoch_{}lr_{}bs_{}L1_{}L2".format(epoch, learningRate, batchSize, firstLayerSize, secondLayerSize)
            DoubleLayerTrainAndTest(modelName, epoch, learningRate, batchSize, firstLayerSize, secondLayerSize)

C:\Users\j.teoh\AppData\Local\miniconda3\envs\tflite-facial-expression\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


loading image
Epoch 1/13
26/26 [==============================] - 11s 293ms/step - loss: 2.1178 - accuracy: 0.1352 - val_loss: 2.0797 - val_accuracy: 0.1039
Epoch 2/13
26/26 [==============================] - 7s 284ms/step - loss: 2.0797 - accuracy: 0.1311 - val_loss: 2.0796 - val_accuracy: 0.1039
Epoch 3/13
26/26 [==============================] - 7s 288ms/step - loss: 2.0790 - accuracy: 0.1319 - val_loss: 2.0796 - val_accuracy: 0.1039
Epoch 4/13
26/26 [==============================] - 7s 283ms/step - loss: 2.0786 - accuracy: 0.1360 - val_loss: 2.0796 - val_accuracy: 0.1234
Epoch 5/13
26/26 [==============================] - 7s 281ms/step - loss: 2.0783 - accuracy: 0.1474 - val_loss: 2.0795 - val_accuracy: 0.1234
Epoch 6/13
26/26 [==============================] - 7s 278ms/step - loss: 2.0780 - accuracy: 0.1474 - val_loss: 2.0795 - val_accuracy: 0.1234
Epoch 7/13
26/26 [==============================] - 7s 281ms/step - loss: 2.0777 - accuracy: 0.1474 - val_loss: 2.0796 - val_accuracy

### Reference

In [ ]:
# Default Train and Test function
def TrainAndTest(modelName):
  """Train and test a model. This function's input should be customized to accept different parameters, function body should be customized to these parameters to train models of variety of paramters.

  Args:
      modelName (str): name of model
  """
  pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  #-------------------------------------------------
  # Customizing layers
  #-------------------------------------------------
  # Adding 3 more layers to output side
  final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  model = keras.Model(inputs = base_input, outputs = final_output)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

  # ------------------ Training Model ------------------
  time_callback = TimeHistory()
  dataSet = mainTrainSet.copy()

  # Initializing model fit params
  batchSize = 4
  imgPerIter = batchSize*8

  count = 0
  while(len(dataSet) != 0):
    croppedList = []
    try:
      print("loading image") # Crop and load images in
      croppedList = CropData(dataSet, imgPerIter)
      label, data = LoadImages(TRAINIMAGEPATH, croppedList)
      
      # Training model
      model.fit(data, label, epochs = 11, batch_size = batchSize, callbacks = [time_callback])
      print(time_callback.AverageTime())
    except RuntimeError as e:
      print(e)
    
    # Saving weights
    model.save_weights('{}.h5'.format(modelName))
    
    # Print summary of current iteration
    count = count + len(croppedList)
    timeLeft = len(dataSet)/imgPerIter * time_callback.AverageTime()
    print("trained image count: ", count)
    print("Images Left: ", len(dataSet))
    print("Estimated completion time: ", str(timedelta(seconds=timeLeft)))
    
    reset_keras()
  print("---------------- {} Training Completed ------------------".format(modelName))

  # ---------------- Testing Model -----------------
  # initialise image names and label
  testSetData = mainTrainSet.copy()

  # test model
  loss, accuracy = TestModel(model, mainTrainSet.copy())
  
  # Saving test informations
  modelSummary = '''
  model name: {}
  average lost: {}
  average accuracy: {}
            '''.format(modelName, loss, accuracy)
      
  print("==========FINISH TESTING===========\n{}".format(modelSummary))
  
  file = open("training_summaries.txt", "a")
  file.write(modelSummary + "\n\n")